In [1]:
                             
import os
import pandas as pd
from sqlalchemy import create_engine, text
                                      
from sqlalchemy import Integer, String, Text, TIMESTAMP, Boolean, VARCHAR, Float
from dotenv import load_dotenv
import logging
import numpy as np
import warnings
import time                    

In [2]:
                        
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
pd.set_option('display.max_rows', 15)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
                                                 
                                                           
                                                                          

In [3]:
                                                 
logging.info("Cargando variables de entorno...")
dotenv_path = '/home/nicolas/Escritorio/workshops/workshop_2/env/.env'                         
load_dotenv(dotenv_path=dotenv_path)

DB_USER = os.getenv('POSTGRES_USER')
DB_PASSWORD = os.getenv('POSTGRES_PASSWORD')
DB_HOST = os.getenv('POSTGRES_HOST')
DB_PORT = os.getenv('POSTGRES_PORT')
DB_NAME = os.getenv('POSTGRES_DB')
SOURCE_TABLE_NAME = 'spotify_dataset'                                  
CLEAN_TABLE_NAME = 'spotify_dataset_clean'                             
CHUNK_SIZE = 40000                                                      

engine = None
if not all([DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME]):
    logging.error("Faltan variables de entorno para la base de datos en " + dotenv_path)
    raise ValueError("Variables de entorno incompletas.")
else:
    try:
        db_url = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
        engine = create_engine(db_url)
        logging.info(f"Motor SQLAlchemy creado para la base de datos '{DB_NAME}'.")
    except Exception as e:
        logging.error(f"Error al crear el motor SQLAlchemy: {e}")
        raise

2025-04-11 12:29:21,685 - INFO - Cargando variables de entorno...
2025-04-11 12:29:21,718 - INFO - Motor SQLAlchemy creado para la base de datos 'artists'.


In [4]:
                                                   
df_spotify = None                     
if engine:
    try:
        logging.info(f"Leyendo datos desde la tabla original '{SOURCE_TABLE_NAME}'...")
        query = f'SELECT * FROM "{SOURCE_TABLE_NAME}"'
                                                                            
                                                                                                     
                                                                    
        df_spotify = pd.read_sql_query(query, con=engine)
        logging.info(f"Datos originales cargados exitosamente desde '{SOURCE_TABLE_NAME}'. Filas: {len(df_spotify)}")
    except Exception as e:
        logging.error(f"Error al leer la tabla original '{SOURCE_TABLE_NAME}': {e}")
        raise
else:
    logging.error("No se pudo crear el engine. No se pueden cargar los datos.")
    raise ValueError("Engine no disponible.")

2025-04-11 12:29:21,727 - INFO - Leyendo datos desde la tabla original 'spotify_dataset'...
2025-04-11 12:29:22,097 - INFO - Datos originales cargados exitosamente desde 'spotify_dataset'. Filas: 40000


In [5]:
                               

                                       
if df_spotify is not None:
    logging.info(f"Creando copia del DataFrame para limpieza: '{CLEAN_TABLE_NAME}'")
                                                                    
    spotify_dataset_clean = df_spotify.copy()
else:
    logging.error("No se puede iniciar la limpieza porque 'df_spotify' no está cargado.")
    raise ValueError("DataFrame 'df_spotify' original no disponible para copiar.")

2025-04-11 12:29:22,104 - INFO - Creando copia del DataFrame para limpieza: 'spotify_dataset_clean'


In [6]:
                                                   
if spotify_dataset_clean is not None:
    logging.info("--- Iniciando Limpieza del DataFrame Spotify ---")
    initial_rows = len(spotify_dataset_clean)
    logging.info(f"Filas iniciales: {initial_rows}")

2025-04-11 12:29:22,117 - INFO - --- Iniciando Limpieza del DataFrame Spotify ---
2025-04-11 12:29:22,119 - INFO - Filas iniciales: 40000


In [7]:
                                    
    num_duplicados_before = spotify_dataset_clean.duplicated().sum()
    if num_duplicados_before > 0:
        spotify_dataset_clean.drop_duplicates(inplace=True)
        logging.info(f"Se eliminaron {num_duplicados_before} filas duplicadas.")
    else:
        logging.info("No se encontraron filas duplicadas.")
    rows_after_duplicates = len(spotify_dataset_clean)

2025-04-11 12:29:22,216 - INFO - Se eliminaron 195 filas duplicadas.


In [8]:
                                                      
                                                                                     
                            
    cols_to_check_nulls = ['artists', 'album_name', 'track_name']
    nulls_before_drop = spotify_dataset_clean[cols_to_check_nulls].isnull().sum().sum()
    if nulls_before_drop > 0:
        spotify_dataset_clean.dropna(subset=cols_to_check_nulls, inplace=True)
        rows_dropped_nulls = rows_after_duplicates - len(spotify_dataset_clean)
        logging.info(f"Se eliminaron {rows_dropped_nulls} filas debido a nulos en {cols_to_check_nulls}.")
    else:
        logging.info(f"No se encontraron nulos en las columnas críticas {cols_to_check_nulls}.")

2025-04-11 12:29:22,238 - INFO - No se encontraron nulos en las columnas críticas ['artists', 'album_name', 'track_name'].


In [9]:
                                     
    logging.info("Eliminando espacios en blanco iniciales/finales de columnas de texto...")
    object_columns = spotify_dataset_clean.select_dtypes(include=['object']).columns
    cols_stripped = []
    for col in object_columns:
        if spotify_dataset_clean[col].notnull().any():
            try:
                spotify_dataset_clean[col] = spotify_dataset_clean[col].str.strip()
                cols_stripped.append(col)
            except AttributeError:
                logging.warning(f"No se pudo aplicar .str.strip() a la columna '{col}'.")
    if cols_stripped:
        logging.info(f"Espacios eliminados en columnas: {cols_stripped}")

2025-04-11 12:29:22,246 - INFO - Eliminando espacios en blanco iniciales/finales de columnas de texto...
2025-04-11 12:29:22,316 - INFO - Espacios eliminados en columnas: ['track_id', 'artists', 'album_name', 'track_name', 'track_genre']


In [10]:
                                                        
    logging.info("Optimizando tipos de datos en Pandas...")
    try:
        for col in spotify_dataset_clean.select_dtypes(include=['object']).columns:
             spotify_dataset_clean[col] = spotify_dataset_clean[col].astype('string')
        logging.info("Columnas de texto convertidas a tipo 'string' de Pandas.")
    except Exception as e:
        logging.warning(f"No se pudieron convertir todas las columnas a 'string': {e}")

    logging.info("--- Limpieza del DataFrame Finalizada ---")
    logging.info(f"Filas restantes: {len(spotify_dataset_clean)}")

2025-04-11 12:29:22,325 - INFO - Optimizando tipos de datos en Pandas...
2025-04-11 12:29:22,375 - INFO - Columnas de texto convertidas a tipo 'string' de Pandas.
2025-04-11 12:29:22,376 - INFO - --- Limpieza del DataFrame Finalizada ---
2025-04-11 12:29:22,377 - INFO - Filas restantes: 39805


In [11]:
                                                
    logging.info("\n--- Verificación del DataFrame Limpio ('spotify_dataset_clean') ---")
    print("\nPrimeras filas del DataFrame limpio:")
    display(spotify_dataset_clean.head())
    print("\nInformación del DataFrame limpio (tipos en Pandas):")
    spotify_dataset_clean.info()
    print("\nConteo de nulos por columna en el DataFrame limpio:")
    display(spotify_dataset_clean.isnull().sum())
    print("\nConteo de duplicados restantes:")
    print(spotify_dataset_clean.duplicated().sum())

2025-04-11 12:29:22,384 - INFO - 
--- Verificación del DataFrame Limpio ('spotify_dataset_clean') ---



Primeras filas del DataFrame limpio:


,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Soundtrack),Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic



Información del DataFrame limpio (tipos en Pandas):
<class 'pandas.core.frame.DataFrame'>
Index: 39805 entries, 0 to 39999
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          39805 non-null  string 
 1   artists           39805 non-null  string 
 2   album_name        39805 non-null  string 
 3   track_name        39805 non-null  string 
 4   popularity        39805 non-null  int64  
 5   duration_ms       39805 non-null  int64  
 6   explicit          39805 non-null  bool   
 7   danceability      39805 non-null  float64
 8   energy            39805 non-null  float64
 9   key               39805 non-null  int64  
 10  loudness          39805 non-null  float64
 11  mode              39805 non-null  int64  
 12  speechiness       39805 non-null  float64
 13  acousticness      39805 non-null  float64
 14  instrumentalness  39805 non-null  float64
 15  liveness          39805 non-null  float

track_id          0
artists           0
album_name        0
track_name        0
popularity        0
                 ..
liveness          0
valence           0
tempo             0
time_signature    0
track_genre       0
Length: 20, dtype: int64


Conteo de duplicados restantes:
0


In [12]:
                                                                                       
if spotify_dataset_clean is not None and engine is not None:

                                                       
    sql_types = {
        'track_id': Text(),                              
        'artists': Text(),                               
        'album_name': Text(),                            
        'track_name': Text(),                            
        'popularity': Integer(),                    
        'duration_ms': Integer(),                   
        'explicit': Boolean(),                     
        'danceability': Float(),                                       
        'energy': Float(),                          
        'key': Integer(),                           
        'loudness': Float(),                        
        'mode': Integer(),                          
        'speechiness': Float(),                     
        'acousticness': Float(),                    
        'instrumentalness': Float(),                  
        'liveness': Float(),                        
        'valence': Float(),                         
        'tempo': Float(),                           
        'time_signature': Integer(),                  
        'track_genre': Text()                            
    }

    logging.info(f"Cargando DataFrame limpio a la tabla '{CLEAN_TABLE_NAME}' en chunks de {CHUNK_SIZE}...")
    start_upload_time = time.time()
    try:
                                                                                                
                                                                                        
        spotify_dataset_clean.to_sql(
            CLEAN_TABLE_NAME,
            con=engine,
            if_exists='replace',
            index=False,
            method='multi',
            dtype=sql_types,
            chunksize=CHUNK_SIZE                                    
        )
        end_upload_time = time.time()
        logging.info(f"DataFrame limpio cargado exitosamente en la tabla '{CLEAN_TABLE_NAME}' en {end_upload_time - start_upload_time:.2f} segundos.")

                                                
        logging.info(f"Verificando número de filas en la tabla '{CLEAN_TABLE_NAME}'...")
        with engine.connect() as connection:
            query_count = text(f'SELECT COUNT(*) FROM "{CLEAN_TABLE_NAME}"')
            result = connection.execute(query_count)
            num_db_clean_rows = result.scalar_one()

        logging.info(f"Número de filas en la tabla limpia '{CLEAN_TABLE_NAME}': {num_db_clean_rows}")
        logging.info(f"Número de filas en el DataFrame limpio: {len(spotify_dataset_clean)}")

        if len(spotify_dataset_clean) == num_db_clean_rows:
            logging.info("¡Verificación de carga exitosa!")
        else:
            logging.warning("Discrepancia en el número de filas entre el DataFrame limpio y la tabla cargada.")

    except Exception as e:
        logging.error(f"Error al cargar el DataFrame limpio en la base de datos: {e}")

elif spotify_dataset_clean is None:
     logging.error("No se puede cargar la tabla limpia porque el DataFrame 'spotify_dataset_clean' no está definido.")
elif engine is None:
     logging.error("No se puede cargar la tabla limpia porque la conexión a la base de datos (engine) no está definida.")

logging.info("--- Proceso de Limpieza y Carga Finalizado para Spotify ---")

2025-04-11 12:29:22,507 - INFO - Cargando DataFrame limpio a la tabla 'spotify_dataset_clean' en chunks de 40000...
2025-04-11 12:29:35,447 - INFO - DataFrame limpio cargado exitosamente en la tabla 'spotify_dataset_clean' en 12.94 segundos.
2025-04-11 12:29:35,448 - INFO - Verificando número de filas en la tabla 'spotify_dataset_clean'...
2025-04-11 12:29:35,452 - INFO - Número de filas en la tabla limpia 'spotify_dataset_clean': 39805
2025-04-11 12:29:35,452 - INFO - Número de filas en el DataFrame limpio: 39805
2025-04-11 12:29:35,452 - INFO - ¡Verificación de carga exitosa!
2025-04-11 12:29:35,453 - INFO - --- Proceso de Limpieza y Carga Finalizado para Spotify ---
